#installing necessary library 
!pip install -q transformers
!pip install transformers
!pip install datasets
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [3]:
# defining model and processory 
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#calling librispeech dataset
librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

Reusing dataset librispeech_asr (C:\Users\prash\.cache\huggingface\datasets\librispeech_asr\clean\2.1.0\468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1)


In [5]:
#taking random dataset for evaluating 
audio_input, sample_rate = sf.read(librispeech_samples_ds[10]["file"])


In [11]:
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

In [12]:
# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

In [13]:
#generating output 
transcription = processor.decode(predicted_ids[0])
print(transcription)

NEAR THE FIRE AND THE ORNAMENTS FRED BROUGHT HOME FROM INDIA ON THE MANTELBOARD


In [21]:
target_transcription = "NEAR THE FIRE AND THE ORNAMENTS FRED BROUGHT HOME FROM INDIA ON THE MANTELBOARD"

In [22]:
with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

In [23]:
#comparing with loss with respect to audio 
loss = model(input_values, labels=labels).loss
loss.backward()

In [24]:
print(loss)

tensor(50.5512, grad_fn=<SumBackward0>)
